# **Part I - Google Trends**

In [2]:
%pip install plotly
%pip install pytrends

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import dask 

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from datetime import datetime, date
import time

import yfinance as yf

from pytrends.request import TrendReq

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests


import os
import glob
import pprint as pp
from itertools import combinations

import warnings
warnings.filterwarnings("ignore")


We decide to study the Granger causal effect of different external sources of information on the variation in Bitcoin prices. The chosen external sources are given as follow.

#### **1. Google Trends - Interest over time** : 
   
This index (between 0 and 100) gives an overview of the popularity on the Internet based on the number of Google searches for a specific keyword. 

The goal here is to start from a collection of keywords. 

Then, we identify if any word or combination of word granger causes the variations in the Bitcoin's price, by looking at the evolution of Bitcoin's Close price and the corresponding logreturns. 

In order to maximize our chances to find any keyword which might help in the prediction of the Bitcoin's price, we decided to take a brute-force approach. The idea is as follows : 
- Define a collection of potential keywords 
- Try all possible combinations of keywords 
- If more than 2 keywords are studied, aggregate the indexes using either mean or max of the studied keywords' indexes. 

## **Google Trends**

**Interest over time**

Numbers represent search interest relative to the highest point on the chart for the given region and time. A value of 100 is the peak popularity for the term. A value of 50 means that the term is half as popular. A score of 0 means there was not enough data for this term.

In [4]:
@dask.delayed
def get_google_trends_data_one_timeframe(kw_list, timeframe) :
    """Scrap data from Google Trends to get daily interest rates for a given list of keywords over a specific timeframe.""" 

    # Connect to Google
    pytrends = TrendReq(hl='en-US', tz=360)

    #Build payload
    pytrends.build_payload(kw_list=kw_list, timeframe=timeframe)
    df = pytrends.interest_over_time()
    df.drop(columns=['isPartial'], inplace=True)

    return df 

def get_google_trends_data(kw_list, start_date, end_date) :
    """Scrap data from Google Trends to get daily interest rates for a given list of keywords for a desired time period."""

    #Create timeframes of 6 months from start_date to end_date 
    years = [i for i in range(start_date.year, end_date.year+1)]

    timeframes = []
    for year in years : 
        timeframes.append(f"{year}-01-01 {year}-06-30")
        if year != end_date.year :
            timeframes.append(f"{year}-07-01 {year}-12-31")
        else :
            timeframes.append(f"{year}-07-01 {end_date.year}-{end_date.month}-{end_date.day}")
    
    #Make API requests on each timeframe and concatenate the results
    allpromises = [get_google_trends_data_one_timeframe(kw_list, timeframe) for timeframe in timeframes]
    alldata=dask.compute(allpromises)[0]
    data = pd.concat(alldata)

    return data    

def main_google_trends() :
    """Main function to call for collecting data from Google Trends. Note that the length of keywords is limited to 5."""
    start_date = date(year=2019, month=1, day=1)
    end_date = datetime.today()

    kw_list = ["Bitcoin", "btc", "btc price", "coinbase", "trading", "crypto", "blockchain", "bitcoin news", "cryptocurrency trends",
           "altcoins", "bitcoin wallet", "bitcoin investment", "cryptocurrency", "bitcoin adoption", "bitcoin technology", "bitcoin transactions"]
    kw_lists = np.array_split(kw_list, np.ceil(len(kw_list) / 5))

    save_dir = "data/raw/google_trends/"
    
    df_list = []
    for index, kw in enumerate(kw_lists):
        data = get_google_trends_data(kw, start_date, end_date)
        df_list.append(data)

    df = pd.concat(df_list, axis=1)

    #Save file 
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    name_file = "keywords_data"
    df.to_parquet(save_dir+name_file+".parquet", use_deprecated_int96_timestamps=True, compression="brotli")

In [6]:
%timeit 
main_google_trends()

In [7]:
gt = pd.read_parquet("data/raw/google_trends/keywords_data.parquet")
fig = px.line(gt, x=gt.index, y=gt.columns, title='Trends for Each Keyword Over Time')
fig.show()

Looking at this dirty plot, we can see that some keywords don't have a high index over the whole timeframe studied. Therefore, we decided to filter out some of them in order to keep the most significant ones. 
We decided to remove keywords whose max index is lower than 70. 

In [8]:
def clean_plot_google_trends_data(path_file) :
    """Clean Google Trends data retrieved from the API."""

    df = pd.read_parquet(path_file) 
    df.index = pd.to_datetime(df.index, utc=True)

    #Retrieve max value for each col 
    max_vals = df.max(axis=0)
    cols_to_keep = max_vals[max_vals > 70].index.tolist()
    df = df[cols_to_keep]

    #Change the name of the index col 
    df = df.rename_axis(index='Date')
    
    #Plot trends for each keyword
    fig = make_subplots(rows=3, cols=3,
                        subplot_titles=(cols_to_keep), shared_yaxes=True)

    for index, column in enumerate(df.columns) : 
        row = (index // 3) + 1
        col = (index % 3) + 1
        fig.add_trace(go.Scatter(x = df.index, y = df[column], showlegend=False), row=row, col=col)

    fig.update_layout(height=700)
    fig.show()

    return df

path_file = "data/raw/google_trends/keywords_data.parquet"
gt = clean_plot_google_trends_data(path_file)

Now, we create a list containing all possible combinations of selected keywords. 

In [9]:
def combinations_keywords(kw_list) : 
    all_combinations = []
    for r in range(1, len(kw_list) + 1):
        all_combinations.extend(list(comb) for comb in combinations(kw_list, r))
    return all_combinations

comb_kw = combinations_keywords(gt.columns)
print(len(comb_kw))

127


## **Bitcoin historical data**

In [10]:
def load_crypto_data(name, start) :
  """Get historical data from Yahoo Finance API for a specific cryptocurrency.
     Clean and transform to logreturns. 
  """
  #Get data
  ticker = yf.Ticker(name)
  df = ticker.history(start=start)

  #Keep desired columns 
  df = df[['Close']]
  
  #Compute logreturns
  df['Logrets'] = np.log(df["Close"]).diff()
  df.dropna(subset="Logrets", inplace=True)
  
  #Plot price and logreturns  
  fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Variation of {name} Logreturns", f"Variation of {name} Prices"))
  fig.add_trace(go.Scatter(x = df.index, y = df.Logrets, showlegend=False), row=1, col=1)
  fig.add_trace(go.Scatter(x = df.index, y = df.Close, showlegend=False), row=1, col=2)
  fig.show()

  return df

btc = load_crypto_data("BTC-USD", "2018-12-31")

#### ✍🏻 **Analysis of bitcoin**

## **Merge DataFrames and save the final file**

Let's remind that we have created a list containing all possible combinations of keywords. 

For each subset containing more than 2 keywords, we will create 2 aggregated indexes (mean and max). We will use them independently in the next part to determine if these indexes, containing information about several keywords at the same time, help in the prediction of the Bitcoin's price variation. 

In [11]:
from sklearn.preprocessing import StandardScaler

@dask.delayed
def merge_crypto_google_trends_one_file(crypto_df, google_trend_df, keyword_list, dest_dir) :
    """Retrieve columns from keyword_list, compute aggregated indexes (if needed). 
       Merge crypto historical data and aggregated Google Trends data. 
       Save results to file. 
    """
    gt = google_trend_df[keyword_list]
    
    #Compute aggregated index 
    if len(keyword_list) > 1 :
        gt["Index_mean"] = gt.mean(axis=1)
        #gt["Index_max"] = gt[gt.columns[:-1].values].max(axis=1)
        gt.drop(columns=keyword_list, inplace=True)

    #Merge
    crypto_gt = crypto_df.merge(gt, how="right", on="Date")

    #Save as parquet 
    if not os.path.isdir(dest_dir):
        os.makedirs(dest_dir)

    name_file = "_".join(keyword_list)
    crypto_gt.to_parquet(dest_dir+name_file, use_deprecated_int96_timestamps=True, compression="brotli")


def merge_crypto_google_trends(crypto_df, google_trend_df, comb_kw) :
    
    dest_dir = "data/clean/crypto_google_trends/"

    allpromises=[merge_crypto_google_trends_one_file(crypto_df, google_trend_df, keyword_list, dest_dir) for keyword_list in comb_kw]
    alldata=dask.compute(allpromises)[0]

merge_crypto_google_trends(btc, gt, comb_kw)

## **Granger Causality test**

**Granger causality test** is carried out only on stationary data hence we first need to check if data is stationary.

**ADF test** is a popular statistical test for checking whether the Time Series is stationary or not which works based on the unit root test. The number of unit roots present in the series indicates the number of differencing operations that are required to make it stationary.

- *Null Hypothesis (H0):* Series has a unit root and is non-stationary.
- *Alternative Hypothesis (H1):* Series has no unit root and is stationary.

First, we check if logreturns are stationary (which should be the case).
It will avoid us to make the test for each file.

In [12]:
def is_stationary(df, col_name):
    result = adfuller(df[col_name])
    return result[1] < 0.05

#Open an arbitrary file 
df = pd.read_parquet("data/clean/crypto_google_trends/Bitcoin")
is_stationary(df, "Logrets")

True

In [13]:
"""
def check_and_drop_nonstationary_oneKw(df) :
    col_name = df.columns[-1]
    return df if is_stationary(df, col_name) else differentiate(df, col_name)

def check_and_drop_nonstationary(df) :
    not_stationary = 0 

    for col_name in ["Index_mean", "Index_max"] : 
        if not is_stationary(df, col_name) : 
            df = differentiate(df, col_name)
            if df is None : 
                print(f"{col_name} not stationary after differencing.")
                df = df.drop(columns=[col_name])
                not_stationary += 1           
    return df if not_stationary < 2 else None
"""

def differentiate(df, col_name) : 
    df[col_name] = df[col_name].diff()
    df = df.dropna(subset=[col_name])
    return None if not is_stationary(df, col_name) else df 

def check_and_drop_nonstationary(df, col_name) :
    return df if is_stationary(df, col_name) else differentiate(df, col_name)


@dask.delayed
def call_check_and_drop(path_file) : 
    dir_name = "data/clean/crypto_google_trends\\"
    file_name = path_file.replace(dir_name, "")

    df = pd.read_parquet(path_file)

    if len(file_name.split("_")) == 1 :
        df = check_and_drop_nonstationary(df, df.columns[-1])
    else : 
        #df = check_and_drop_nonstationary(df)
        df = check_and_drop_nonstationary(df, "Index_mean")

    if (df is None and os.path.exists(path_file)):
        print(f"{file_name} removed.")
        os.remove(path_file)

def prepare_data_causality_test() :
    allfiles = glob.glob("data/clean/crypto_google_trends/*")
    allpromises=[call_check_and_drop(path_file) for path_file in allfiles]
    alldata=dask.compute(allpromises)[0]

In [14]:
prepare_data_causality_test()

Now that we made all time series of interest are stationary, we want to study if there is any granger causality between the index of interest and Bitcoin's logreturns. 

To do so, we use the Granger Causality Test to answer the question : *Does the studied index cause Bitcoin's variations?*.

- *Null Hypothesis (H0)* : Interest do not granger cause Bitcoin's variations.
- *Alternative Hypothesis (H1)* : Interest granger cause Bitcoin's variations.


In [15]:
def granger_causality_tests(df, source, target, maxlag, filename) : 

    #Make the different tests
    results = grangercausalitytests(df[[target, source]], maxlag=maxlag, verbose=False)

    #Store the p-values for each test and for each lag
    name_tests = ["ssr_ftest", "ssr_chi2test", "lrtest"] 
    name_tests = ["ssr_ftest", "ssr_chi2test", "lrtest"] 
    avg_pvals = {
        i: np.mean([results[i][0][name][1] for name in name_tests])
        for i in range(1, maxlag + 1)
    }

    """
    if filename == source : 
        column_name = f"{source}"
    else : 
        column_name = f"{source}-{filename}"
    """

    pvals_df = pd.DataFrame.from_dict(avg_pvals, orient='index', columns=[f"{filename.replace('_', ' ')}"])
    
    return pvals_df


@dask.delayed
def call_granger_causality_tests_one_file(path_file, maxlag) : 

    dir_name = "data/clean/crypto_google_trends\\"
    file_name = path_file.replace(dir_name, "")

    df = pd.read_parquet(path_file)
    
    if len(file_name.split("_")) == 1 :
        return granger_causality_tests(df, file_name, "Logrets", maxlag, file_name)
    else : 
        pvals_df = None
        pvals_df = granger_causality_tests(df, "Index_mean", "Logrets", maxlag, file_name) 

        """if "Index_mean" in df.columns :
            pvals_df = granger_causality_tests(df, "Index_mean", "Logrets", maxlag, file_name) 
        if "Index_max" in df.columns :
            pvals_df2 = granger_causality_tests(df, "Index_max", "Logrets", maxlag, file_name)
            pvals_df = pd.concat([pvals_df, pvals_df2], axis=1)"""
        
        return pvals_df

def main_granger_causal_effect(save_dir) :
    
    allfiles = glob.glob("data/clean/crypto_google_trends/*")
    allpromises = [call_granger_causality_tests_one_file(path_file, maxlag=30) for path_file in allfiles]
    alldata = dask.compute(allpromises)[0]
    data = pd.concat(alldata, axis=1)

    #Save as parquet 
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    name_file = "google_trends_logrets"
    data.to_parquet(save_dir+name_file, use_deprecated_int96_timestamps=True, compression="brotli")

main_granger_causal_effect(save_dir="data/raw/pvals_granger_test/")

## **Results**

We have stored the averaged pvalues over the 3 tests for each lag and each potential granger cause. 

Now, we clean the former dataframe to extract combination of keywords which have a granger causal effect, and extract which ones have the most significant effect. 

In [16]:
def clean_visualize_granger_causal_effect(path_file, save_dir) :
    p_vals_df = pd.read_parquet(path_file)

    #Remove columns whose all pvalues are >= 0.05 
    max_pvals = p_vals_df.min(axis=0)
    cols_to_keep = max_pvals[max_pvals < 0.05].index.values
    print(f"{len(p_vals_df.columns) - len(cols_to_keep)}/{len(p_vals_df.columns)} columns discarded (not significant).")
    p_vals_df = p_vals_df[cols_to_keep]

    #Visualize some statistics
    fig = make_subplots(rows=1, cols=2, 
                   subplot_titles=("Average p-value", "Maximal p-value"))

    # Check mean over all lags
    mean_pvals = p_vals_df.mean(axis=0)
    trace_mean = px.histogram(x=mean_pvals, nbins=40, title="Distribution of average p-value")

    # Check max over all lags
    max_pvals = p_vals_df.max(axis=0)
    trace_max = px.histogram(x=max_pvals, title="Distribution of maximal p-value")

    # Add the traces to the subplot
    fig.add_trace(trace_mean.data[0], row=1, col=1)
    fig.add_trace(trace_max.data[0], row=1, col=2)

    fig.update_layout(title_text="Distribution for each index over all lags")
    fig.show()

    #Remove columns with average p-value >= 0.3
    mean_to_keep = mean_pvals[mean_pvals < 0.3].index.tolist()
    print(f"{len(p_vals_df.columns) - len(mean_to_keep)}/{len(p_vals_df.columns)} columns discarded (average).")

    #Remove columns with max p-value >= 0.9
    max_to_keep = max_pvals[max_pvals < 0.9].index.tolist()
    print(f"{len(p_vals_df.columns) - len(max_to_keep)}/{len(p_vals_df.columns)} columns discarded (max).")

    to_keep = set(mean_to_keep + max_to_keep)
    p_vals_df = p_vals_df[to_keep]

    #Check at which lag, on average, google trends index have the strongest granger causal effect on log_returns.
    avg_lag = p_vals_df.mean(axis=1)
    fig = px.line(x=avg_lag.index, y=avg_lag.values, title="Average p-values over all indexes at each lag")
    fig.add_trace(go.Scatter(   x=avg_lag.index,
                                y=[0.05] * len(avg_lag),
                                mode='lines',
                                line=dict(color='red', width=2, dash='dash'),
                                showlegend=False))
    fig.show()

    #Save as parquet 
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    name_file = "google_trends_logrets"
    p_vals_df.to_parquet(save_dir+name_file, use_deprecated_int96_timestamps=True, compression="brotli")

In [17]:
clean_visualize_granger_causal_effect(path_file = "data/raw/pvals_granger_test/google_trends_logrets",
                                      save_dir = "data/clean/pvals_granger_test/")

69/127 columns discarded (not significant).


11/58 columns discarded (average).
30/58 columns discarded (max).


Most of indexes seem to granger cause variations of Bitcoin after the 12 days. Thus, the predictability power of an indexes after lag 12 might not be super interesting to determine which index is the most significant. Therefore, we decided to focus on finding keywords whose scores are the most statistically significant only for lags in [1,12]. 

For each lag in [1,12], we find the top five index with the smallest p-value obtained at the Granger causality test, which might be the most likely to granger cause Bitcoin's logreturns. 

In [111]:
def top_index_per_lag(nb_kw, start_lag, end_lag, height, path) : 

    p_vals_df = pd.read_parquet(path)

    #Remove lags which don't have any significant index
    p_vals_df['min_pval'] = p_vals_df.min(axis=1)
    p_vals_df = p_vals_df[p_vals_df["min_pval"] < 0.05].drop(columns=['min_pval'])

    lags = p_vals_df.index.to_numpy()
    lags = lags[(lags <= end_lag) & (lags >= start_lag)]

    fig = make_subplots(rows=len(lags), cols=1, subplot_titles=[f"Lag {i}" for i in lags], shared_xaxes=True)

    cpt = 1
    all_res = []
    for lag in lags : 
        vals = p_vals_df.loc[p_vals_df.index == lag].transpose().sort_values(by=lag)
        vals = vals.loc[vals[lag] < 0.05]

        vals_df = vals.head(3)
        all_res.append(vals_df)

        vals = vals.head(nb_kw).sort_values(by=lag, ascending=False)
        trace = px.bar(x=vals[lag], y=vals.index.tolist(), orientation='h')
        fig.add_trace(trace.data[0], row=cpt, col=1)
        cpt += 1

    fig.update_layout(height = height, width = 800, title_text="Most significant keywords in predicting Bitcoin's logreturns",
                    margin=dict(l=300, r=200))
    fig.update_yaxes(automargin='left')
    fig.update_xaxes(range=[0, 0.05])


    fig.show()
    results = pd.concat(all_res, axis=1)

    return results

In [115]:
res_1_12 = top_index_per_lag(nb_kw = 5, start_lag = 1, end_lag = 12, height = 1000, path="data/clean/pvals_granger_test/google_trends_logrets")

In [116]:
res_13_30 = top_index_per_lag(nb_kw = 5, start_lag = 13, end_lag = 30, height=2000, path="data/clean/pvals_granger_test/google_trends_logrets")

In [119]:
res

,1,2,10,11,12,13,14,15,16,17,...,21,22,23,24,25,26,27,28,29,30
Bitcoin trading,0.008753,0.032969,0.022170,0.006124,0.003102,0.002450,0.004272,0.005338,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bitcoin btc trading,0.009570,0.039947,0.028194,0.008301,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
btc trading,0.024811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bitcoin cryptocurrency,NaN,NaN,0.031234,0.010279,NaN,0.003013,0.004664,0.004343,0.003498,0.006032,...,0.016083,0.011800,0.004867,0.004072,0.005161,0.007978,0.009717,0.013004,0.018806,0.022791
Bitcoin trading cryptocurrency,NaN,NaN,NaN,NaN,0.003402,NaN,0.005079,NaN,0.003550,0.005986,...,0.015234,0.015598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bitcoin btc trading cryptocurrency,NaN,NaN,NaN,NaN,0.003538,0.003268,NaN,NaN,0.003873,0.006259,...,0.016292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bitcoin btc cryptocurrency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004408,NaN,NaN,...,NaN,0.013755,0.005063,0.004011,0.004650,0.006980,0.008907,0.011494,0.017436,0.021615
btc cryptocurrency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.006936,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bitcoin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.004389,0.004573,0.006534,0.008086,0.010443,0.013851,0.018049


In [123]:
#Merge all results in one dataframe
res = pd.concat([res_1_12, res_13_30], axis = 1)

print(f"Number of indexes with higher granger causal effect over all lags : {len(res)}")

fig = px.bar(res.transpose())
fig.show()

Number of indexes with higher granger causal effect over all lags : 9


**Interesting:** Among 127 possible combinations of keywords, only nine appear in the top 3 words for each lag (3*30=90 possibilities). 

**Evolution of some keyword over time:** trading keywords (Bitcoin trading, Bitcoin btc trading) are significant only at the beginning (quicker impact/trading view/people who know better the sector) / cryptocurrency (more general/curiosity/influence) appear later / even later : bitcoin + bitcoin cryptocurrency 